## **PROJECT - NOTEBOOK #1: Raw Data**

This notebook sets up our data pipeline by configuring the environment, importing essential libraries and create a SQLAlchemy engine, then loading raw data from the CSV file into a Pandas DataFrame, transferring this data into a MySQL database, and verifying the transfer with a simple query.

---

### **Setting Environment**

In [2]:
import os 
print(os.getcwd())

try:
    os.chdir("../../project_etl")

except FileNotFoundError:
    print("""
        FileNotFoundError - The directory may not exist or you might not be in the specified path.
        If this has already worked, do not run this block again, as the current directory is already set to workshop-001.
        """)
    
print(os.getcwd())

d:\U\FIFTH SEMESTER\ETL\project_etl\notebooks
d:\U\FIFTH SEMESTER\ETL\project_etl


### **Importing modules and libraries**

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arshkon/linkedin-job-postings")

print("Path to dataset files:", path)

dataset_dir = path
csv_file = os.path.join(dataset_dir, "postings.csv")

Path to dataset files: C:\Users\sebas\.cache\kagglehub\datasets\arshkon\linkedin-job-postings\versions\13


In [3]:
import pandas as pd
from functions.db_connection.connection import creating_engine
from sqlalchemy import text

### **Ingest Data**

In [9]:
if os.path.exists(csv_file):
    df = pd.read_csv(csv_file)
    print("DataFrame loaded successfully!")
    print("First 5 rows of the dataset:")
    print(df.head())
else:
    print(f"Error: '{csv_file}' not found in {dataset_dir}")
    print("Available files:", os.listdir(dataset_dir))

DataFrame loaded successfully!
First 5 rows of the dataset:
     job_id            company_name  \
0    921716   Corcoran Sawyer Smith   
1   1829192                     NaN   
2  10998357  The National Exemplar    
3  23221523  Abrams Fensterman, LLP   
4  35982263                     NaN   

                                               title  \
0                              Marketing Coordinator   
1                  Mental Health Therapist/Counselor   
2                        Assitant Restaurant Manager   
3  Senior Elder Law / Trusts and Estates Associat...   
4                                 Service Technician   

                                         description  max_salary pay_period  \
0  Job descriptionA leading real estate firm in N...        20.0     HOURLY   
1  At Aspen Therapy and Wellness , we are committ...        50.0     HOURLY   
2  The National Exemplar is accepting application...     65000.0     YEARLY   
3  Senior Associate Attorney - Elder Law / Trusts...